# General material models

In [1]:
import numpy as np

from sigmaepsilon.solid.material import ElasticityTensor
from sigmaepsilon.math.linalg import ReferenceFrame
from sigmaepsilon.solid.material.utils import elastic_stiffness_matrix
from sigmaepsilon.math.logical import isposdef, issymmetric
from sigmaepsilon.solid.material import (
    ElasticityTensor,
    LinearElasticMaterial,
    HuberMisesHenckyFailureCriterion,
)

hooke = elastic_stiffness_matrix(E=210000, NU=0.3)
assert isposdef(hooke)
assert issymmetric(hooke, 1e-8)

yield_strength=355.0

frame = ReferenceFrame(dim=3)
stiffness = ElasticityTensor(hooke, frame=frame, tensorial=False)
failure_model = HuberMisesHenckyFailureCriterion(yield_strength=yield_strength)
material = LinearElasticMaterial(stiffness=stiffness, failure_model=failure_model)

In [2]:
material.calculate_strains(stresses=np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0]))

array([[ 0.00169048, -0.00050714, -0.00050714,  0.        ,  0.        ,
         0.        ]])

In [3]:
material.utilization(stresses=np.random.rand(10, 6))

array([0.00454578, 0.00218466, 0.00447378, 0.00593879, 0.00546288,
       0.00251926, 0.00672043, 0.00349326, 0.00330169, 0.00461547])

In [4]:
material.calculate_stresses(strains=np.random.rand(6)).shape

(1, 6)

In [5]:
material.calculate_stresses(strains=np.random.rand(10, 6)).shape

(10, 6)

In [6]:
stresses=np.array([yield_strength, 0.0, 0.0, 0.0, 0.0, 0.0])
strains = material.calculate_strains(stresses=stresses)
material.utilization(stresses=stresses) * 100

100.0

In [7]:
strains = material.calculate_strains(stresses=np.eye(6) * yield_strength)
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

array([1.        , 1.        , 1.        , 1.73205081, 1.73205081,
       1.73205081])

In [8]:
strains = material.calculate_strains(stresses=np.array([10*yield_strength, 10*yield_strength, 10*yield_strength, 0.0, 0.0, 0.0]))
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses)

array(2.56195691e-15)

Utilization for a single state of strain:

In [9]:
strains = 2*np.random.rand(6)/1000
strains

array([0.00173154, 0.00161796, 0.00027652, 0.00169012, 0.00167926,
       0.00118488])

In [10]:
stresses = material.calculate_stresses(strains=strains)
material.utilization(stresses=stresses) * 100

122.73374966882162

Utilizations for several states with bulk evaluation:

In [11]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000) * 100

array([0.00085395, 0.00134484, 0.00071666, 0.00105298, 0.00071055,
       0.00113623, 0.00121962, 0.00046218, 0.00080682, 0.0010324 ])

In [12]:
material.utilization(stresses=2*np.random.rand(10, 6)/1000).shape

(10,)

In [13]:
material.utilization(stresses=tuple(2*np.random.rand(10)/1000 for _ in range(6))) * 100

array([0.0010681 , 0.001052  , 0.00089471, 0.00078132, 0.00098108,
       0.0005937 , 0.00108656, 0.00080096, 0.00104349, 0.00110625])